<a href="https://colab.research.google.com/github/maddog19527/CS660_Project_3/blob/Mike's-Draft/Project_3_Outlier_Detection_Comparison_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor, LocalOutlierFactor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.tree import plot_tree
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.datasets import fetch_california_housing


housing=fetch_california_housing()
cal_df=pd.DataFrame(data=housing.data, columns=housing.feature_names)
cal_df['Medvalue'] = housing.target
print(cal_df.head())
print(cal_df.shape)

   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  Medvalue  
0    -122.23     4.526  
1    -122.22     3.585  
2    -122.24     3.521  
3    -122.25     3.413  
4    -122.25     3.422  
(20640, 9)


Z-Score Outlier Detection

In [5]:
def ZOutliers(df):
  zdf=df.copy()
  outliers=pd.DataFrame()
  for col in zdf.columns:
    zscore=stats.zscore(zdf[col])
    zdf[f'{col}_zscore']=zscore
    mask=zdf[f'{col}_zscore'].abs()<2.5
    zdf=zdf[mask]
    outliers.drop_duplicates(inplace=True)
    z_score_columns=[col for col in outliers.columns if col.endswith('_zscore')]
    outliers=outliers.drop(columns=z_score_columns)
    outliers.to_csv('outliers.csv', index=False)
  return zdf

IQR Outlier Detection

In [6]:
def OutlierDetection(df):
  clean_df=df.copy()
  outliers=pd.DataFrame()
  for col in clean_df.columns:
    Q1=df[col].quantile(0.25)
    Q3=df[col].quantile(0.75)
    IQR=Q3-Q1
    lower=Q1-1.5*IQR
    upper=Q3+1.5*IQR
    mask=(df[col] >= lower) & (df[col]<=upper)
    clean_df=clean_df[mask]
    outliers[col]=df[col][~mask]
  return clean_df

KNN Outlier Detection

In [7]:
def KNNOutliers(df):
  df1=df.copy()
  combinedoutliers=pd.DataFrame()
  for col in df.columns:
    k=20
    knn=KNeighborsRegressor(n_neighbors=k)
    knn.fit(df1.iloc[:,1:9], df1[col])
    distances,indeces=knn.kneighbors(df1.iloc[:,1:9])
    df1[f'outlierscore for {col}']=np.mean(distances, axis=1)
    threshold=np.percentile(np.mean(distances, axis=1), 95)
    mask=df1[f'outlierscore for {col}']<threshold
    df1=df1[mask]
    # outliersdoc=pd.DataFrame(outliers)
    # outliersdoc.drop_duplicates(inplace=True)
    # outliersdoc.columns=[col]
    # combinedoutliers=pd.concat([combinedoutliers, outliersdoc], ignore_index=True)
  return df1


Running the base Regression Analysis using each outlier detection program for camprison.

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class OutlierHandling(BaseEstimator, TransformerMixin):
  def __init__(self, method='KNN'):
    self.method=method

  def fit(self, X, y=None):
    return self

  def transform(self, X):
    if self.method=='KNN':
      return KNNOutliers(X)
    elif self.method=='Z':
      return ZOutliers(X)
    elif self.method=='IQR':
      return OutlierDetection(X)
    else:
      raise ValueError("Invalid outlier detection method")



In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import GridSearchCV
# def outlierstest(df):
#     X=df.drop(columns=['Medvalue'])
#     y=df['Medvalue']
#     # df['y']=df['Medvalue']
#     x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)
#     pipeline=Pipeline([('outlierhandler', OutlierHandling()), ('model', RandomForestRegressor())])
#     param_grid=[
#             {
#             "model": [RandomForestRegressor()],
#             "outlierhandler__method": ["KNN", "Z", "IQR"],
#             "model__n_estimators"      : [10,20,30],
#             "model__min_samples_split" : [2,4,8],
#             },
#             {
#             'model': [DecisionTreeRegressor()],
#             'outlierhandler__method': ['KNN', 'Z', 'IQR'],
#             'model__max_depth': [10, 20, 30, None],
#             'model__min_samples_split': [2, 5, 10],
#             'model__min_samples_leaf': [1, 2, 4]
#             }]
#     gridclf=GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
#     gridclf.fit(x_train,y_train)
#     print("Best parameters:", gridclf.best_params_)
#     print("Best model:", gridclf.best_estimator_)
# outlierstest(cal_df)